## Shipment Tracking
Using the given dataset, your task is to generate a
flattened dataset consumable by analysts and generate a summary of the dataset given to you.

### Import Libraries

In [21]:
# importing necessary libraries
import pandas as pd
import json
import csv
import datetime as dt

### Read Json data from given file

In [4]:
# import the csv data through pandas
dataFrame = pd.read_json("Swift Assignment 4 - Dataset.json")
dataFrame.head(3)

,highestSeverity,notifications,duplicateWaybill,moreData,trackDetailsCount,trackDetails
0,SUCCESS,"[{'severity': 'SUCCESS', 'source': 'trck', 'co...",False,False,0,"[{'notification': {'severity': 'SUCCESS', 'sou..."
1,SUCCESS,"[{'severity': 'SUCCESS', 'source': 'trck', 'co...",False,False,0,"[{'notification': {'severity': 'SUCCESS', 'sou..."
2,SUCCESS,"[{'severity': 'SUCCESS', 'source': 'trck', 'co...",False,False,0,"[{'notification': {'severity': 'SUCCESS', 'sou..."


### Flatten the data

In [5]:
#flatten data
df =  pd.json_normalize(dataFrame['trackDetails'])
df.head(3)

,0
0,"{'trackingNumber': '391128701026', 'trackingNu..."
1,"{'trackingNumber': '390901883808', 'trackingNu..."
2,"{'trackingNumber': '391128749178', 'trackingNu..."


Storing the data into the flattened_data as list

In [49]:
flattened_data = []
for track_list in dataFrame['trackDetails']:
    for track in track_list:
        # Find the event with event type PU
        pickup_event = next((event for event in track['events'] if event['eventType']=='PU'), None)

        # extract the postal code if PU event is found
        postal_code = pickup_event['address']['postalCode'] if pickup_event and 'address' in pickup_event else ''

        # extract COD payment type else set the payment type Prepaid
        paymet_type = next((handling.get('type', '') for handling in track.get('specialHandlings', []) if handling.get('type') == 'COD'), 'Prepaid')

        flattened_data.append({
        'Tracking number': track['trackingNumber'],
        'Payment type': paymet_type,
        'Pickup Date Time': track['datesOrTimes'][1]['dateOrTimestamp'],
        'Delivery Date Time': track['datesOrTimes'][0]['dateOrTimestamp'],
        'Out for Delivery datetime(s)': [event['timestamp'] for event in track['events'] if event['eventType'] == 'OD'],
        'Shipment weight': track['shipmentWeight']['value'],
        'Pickup Pincode': postal_code,
        'Pickup City': track['shipperAddress']['city'],
        'Pickup State': track['shipperAddress']['stateOrProvinceCode'],
        'Drop Pincode': track['events'][0]['address']['postalCode'],
        'Drop city': track['events'][0]['address']['city'],
        'Drop state': track['events'][0]['address']['stateOrProvinceCode'],
        'Number of Delivery attempts needed':0,
    })

In [50]:
# for to see sample data
flattened_data

[{'Tracking number': '391128701026',
  'Payment type': 'COD',
  'Pickup Date Time': '2020-03-16T15:44:00+05:30',
  'Delivery Date Time': '2020-03-20T13:37:00+05:30',
  'Out for Delivery datetime(s)': [{'$numberLong': '1584679560000'}],
  'Shipment weight': 14,
  'Pickup Pincode': '560048',
  'Pickup City': 'Bangalore',
  'Pickup State': 'KA',
  'Drop Pincode': '122001',
  'Drop city': 'Gurgaon',
  'Drop state': 'HR',
  'Number of Delivery attempts needed': 0},
 {'Tracking number': '390901883808',
  'Payment type': 'Prepaid',
  'Pickup Date Time': '2020-03-06T16:07:00+05:30',
  'Delivery Date Time': '2020-03-09T19:50:00+05:30',
  'Out for Delivery datetime(s)': [{'$numberLong': '1583731200000'},
   {'$numberLong': '1583729700000'}],
  'Shipment weight': 14,
  'Pickup Pincode': '560048',
  'Pickup City': 'Bangalore',
  'Pickup State': 'KA',
  'Drop Pincode': '560034',
  'Drop city': 'Bangalore',
  'Drop state': 'KA',
  'Number of Delivery attempts needed': 0},
 {'Tracking number': '39112

### Transforming the data

In [51]:
# Convert the time format
for item in flattened_data:
    #Convert the datetime to readable format in IST
    item['Pickup Date Time'] = dt.datetime.fromisoformat(item['Pickup Date Time']).astimezone(dt.timezone(dt.timedelta(hours=5, minutes=30))).strftime('%Y-%m-%d %H:%M:%S')
    item['Delivery Date Time'] = dt.datetime.fromisoformat(item['Delivery Date Time']).astimezone(dt.timezone(dt.timedelta(hours=5, minutes=30))).strftime('%Y-%m-%d %H:%M:%S')

    # Calculate the number of days taken to delivery
    days_taken = (dt.datetime.fromisoformat(item['Delivery Date Time']) - dt.datetime.fromisoformat(item['Pickup Date Time'])).days
    item['Days Taken for Delivery'] = days_taken

    # Calculate the number of Delivery Attempts
    item['Number of Delivery attempts needed'] = len([event for event in track['events'] if event['eventType'] in ['OD', 'DL']])


In [53]:
#Create DataFrame
df = pd.DataFrame(flattened_data)
df.head()

,Tracking number,Payment type,Pickup Date Time,Delivery Date Time,Out for Delivery datetime(s),Shipment weight,Pickup Pincode,Pickup City,Pickup State,Drop Pincode,Drop city,Drop state,Number of Delivery attempts needed,Days Taken for Delivery
0,391128701026,COD,2020-03-16 15:44:00,2020-03-20 13:37:00,[{'$numberLong': '1584679560000'}],14.0,560048,Bangalore,KA,122001,Gurgaon,HR,3,3
1,390901883808,Prepaid,2020-03-06 16:07:00,2020-03-09 19:50:00,"[{'$numberLong': '1583731200000'}, {'$numberLo...",14.0,560048,Bangalore,KA,560034,Bangalore,KA,3,3
2,391128749178,Prepaid,2020-03-16 15:44:00,2020-03-19 15:29:00,[{'$numberLong': '1584601200000'}],14.0,560048,Bangalore,KA,380028,Ahmedabad,GJ,3,2
3,390807986805,Prepaid,2020-03-03 16:19:00,2020-03-07 14:24:00,[{'$numberLong': '1583557020000'}],14.0,560048,Bangalore,KA,110088,New Delhi,DL,3,3
4,390948921190,COD,2020-03-09 15:12:00,2020-03-13 14:44:00,[{'$numberLong': '1584072120000'}],14.0,560048,Bangalore,KA,110009,Delhi,DL,3,3


### Output as CSV file

In [54]:
output_csv_path = 'output.csv'
df.to_csv(output_csv_path, index=False)

###     Output Summary Statistics as a CSv

In [60]:
# extract required column for statistics analysis
summary_df = df[['Days Taken for Delivery', 'Number of Delivery attempts needed']]

# Calculate the mean, median mode
mean_value = summary_df.mean()
median_value = summary_df.median()
mode_value = summary_df.mode().iloc[0]

# create dataframe for Summary statistics
Summary = pd.DataFrame({
    'Mean':mean_value,
    'Median':median_value,
    'Mode':mode_value
})

# Output Summary Statistics in csv file
Summary.to_csv('Statistics_Summary.csv')

#### Other way to calculate the summary through pandas

In [62]:
summary_stats = summary_df.describe().transpose()
summary_stats.to_csv('Summary_Description.csv')